In [8]:
import livef1

# session = livef1.get_session(
#     season = 2025,
#     meeting_identifier = "Australian",
#     session_identifier = "Qualifying"
# )

# session.generate()

session = livef1.get_session(
    season = 2025,
    meeting_identifier = "Sakhir",
    session_identifier = "Day 2"
)

session.generate()

21:30:34 - Selected meeting/session is:
	Meeting Offname : FORMULA 1 ARAMCO PRE-SEASON TESTING 2025
	Meeting Name : Pre-Season Testing
	Meeting Circuit Shortname : Sakhir
21:30:34 - Got the meeting.
21:30:34 - Selected meeting/session is:
	Session Name : Day 2
21:30:34 - The session was received successfully.
21:30:34 - Fetching drivers.
21:30:36 - Fetching data : 'SessionStatus'
21:30:36 - Fetching data : 'RaceControlMessages'
21:30:36 - Fetching data : 'TimingData'
21:30:36 - Fetching data : 'CarData.z'
21:30:36 - Fetching data : 'Position.z'
21:30:36 - 'RaceControlMessages' has been fetched and parsed
21:30:36 - 'SessionStatus' has been fetched and parsed
21:30:40 - 'TimingData' has been fetched and parsed
21:30:44 - 'Position.z' has been fetched and parsed
21:30:49 - 'CarData.z' has been fetched and parsed
21:31:05 - Silver tables are being generated.
21:31:06 - 'laps' has been generated and saved to the silver lake. You can access it from 'session.laps'.
21:31:26 - 'carTelemetry' 

In [9]:
df = session.laps.groupby("DriverNo")[["sector1_time","sector2_time","sector3_time"]].min().reset_index()
df["sector1_diff"] = (df["sector1_time"] - df["sector1_time"].min()).dt.total_seconds()
df["sector2_diff"] = (df["sector2_time"] - df["sector2_time"].min()).dt.total_seconds()
df["sector3_diff"] = (df["sector3_time"] - df["sector3_time"].min()).dt.total_seconds()
df["DriverName"] = df["DriverNo"].map(lambda x: session.drivers[x].FullName)
df["TeamColour"] = df["DriverNo"].map(lambda x: f"#{session.drivers[x].TeamColour}")
df["LastName"] = df["DriverNo"].map(lambda x: session.drivers[x].LastName)
df["TeamName"] = df["DriverNo"].map(lambda x: session.drivers[x].TeamName)
df["Tla"] = df["DriverNo"].map(lambda x: session.drivers[x].Tla)
df

,DriverNo,sector1_time,sector2_time,sector3_time,sector1_diff,sector2_diff,sector3_diff,DriverName,TeamColour,LastName,TeamName,Tla
0,10,0 days 00:00:29.009000,0 days 00:00:38.658000,0 days 00:00:22.636000,0.278,0.483,0.355,Pierre GASLY,#0093CC,Gasly,Alpine,GAS
1,12,0 days 00:00:28.965000,0 days 00:00:38.478000,0 days 00:00:22.308000,0.234,0.303,0.027,Andrea Kimi ANTONELLI,#27F4D2,Antonelli,Mercedes,ANT
2,14,0 days 00:00:29.241000,0 days 00:00:38.821000,0 days 00:00:22.626000,0.510,0.646,0.345,Fernando ALONSO,#229971,Alonso,Aston Martin,ALO
3,16,0 days 00:00:28.804000,0 days 00:00:38.220000,0 days 00:00:22.304000,0.073,0.045,0.023,Charles LECLERC,#E80020,Leclerc,Ferrari,LEC
4,18,0 days 00:00:28.881000,0 days 00:00:38.607000,0 days 00:00:22.639000,0.150,0.432,0.358,Lance STROLL,#229971,Stroll,Aston Martin,STR
5,22,0 days 00:00:28.980000,0 days 00:00:39.001000,0 days 00:00:22.738000,0.249,0.826,0.457,Yuki TSUNODA,#6692FF,Tsunoda,Racing Bulls,TSU
6,27,0 days 00:00:29.354000,0 days 00:00:39.126000,0 days 00:00:22.819000,0.623,0.951,0.538,Nico HULKENBERG,#52E252,Hulkenberg,Kick Sauber,HUL
7,30,0 days 00:00:29.092000,0 days 00:00:38.593000,0 days 00:00:22.484000,0.361,0.418,0.203,Liam LAWSON,#3671C6,Lawson,Red Bull Racing,LAW
8,31,0 days 00:00:30.158000,0 days 00:00:39.836000,0 days 00:00:23.077000,1.427,1.661,0.796,Esteban OCON,#B6BABD,Ocon,Haas F1 Team,OCO
9,4,0 days 00:00:28.975000,0 days 00:00:38.175000,0 days 00:00:22.624000,0.244,0.000,0.343,Lando NORRIS,#FF8000,Norris,McLaren,NOR


In [5]:
list(session.get_topic_names().keys())

['SessionInfo',
 'ArchiveStatus',
 'TrackStatus',
 'SessionData',
 'ContentStreams',
 'TyreStintSeries',
 'CarData.z',
 'Position.z',
 'ExtrapolatedClock',
 'SessionStatus',
 'DriverList',
 'TimingDataF1',
 'TopThree',
 'TimingAppData',
 'TimingData',
 'DriverTracker',
 'LapSeries',
 'TimingStats',
 'Heartbeat',
 'WeatherData',
 'WeatherDataSeries',
 'TlaRcm',
 'RaceControlMessages',
 'PitLaneTimeCollection',
 'CurrentTyres',
 'TeamRadio']

In [5]:
df_rcm = session.get_data("RaceControlMessages")
df_rcm_del = df_rcm[(df_rcm["Category"] == "Other") & (df_rcm.Message.str.split(" ").str[0] == "CAR")]

df_rcm_del["deleted_driver"] = df_rcm_del.Message.str.split(" ").str[1]
df_rcm_del["deleted_type"] = df_rcm_del.Message.str.split(" ").str[3]
df_rcm_del["deleted_lap"] = df_rcm_del.apply(lambda x: x.Message.split(" ")[12] if x.deleted_type == "LAP" else x.Message.split(" ")[13] if x.deleted_type == "TIME" else None, axis=1)
df_rcm_del["deleted_lap"] = df_rcm_del.apply(lambda x: x.Message.split(" ")[12] if x.deleted_type == "LAP" else x.Message.split(" ")[13] if x.deleted_type == "TIME" else None, axis=1)

# 1 DriverNo
# 3 Type

# 12 LAP
df_rcm_del.dtypes

SessionKey          int64
timestamp          object
Utc                object
Category           object
Flag               object
Scope              object
Message            object
Sector            float64
deleted_driver     object
deleted_type       object
deleted_lap        object
dtype: object

In [8]:
laps.dtypes


lap_number                  float64
lap_time            timedelta64[ns]
in_pit              timedelta64[ns]
pit_out             timedelta64[ns]
sector1_time        timedelta64[ns]
sector2_time        timedelta64[ns]
sector3_time        timedelta64[ns]
None                         object
speed_I1                     object
speed_I2                     object
speed_FL                     object
speed_ST                     object
no_pits                     float64
lap_start_time      timedelta64[ns]
DriverNo                     object
isDeleted                      bool
deleationMessage             object
lap_start_date       datetime64[ns]
dtype: object

In [5]:
laps.loc[(laps["lap_number"] == 1) & (laps["DriverNo"] == "7"), "lap_time"]

239   0 days 00:02:06.136000
Name: lap_time, dtype: timedelta64[ns]